In [ ]:
# !pip install scikit-surprise
# !pip install snowflake-connector-python
# !pip install "snowflake-connector-python[pandas]"

In [ ]:
print("Hello World")

In [ ]:
import os
import csv
import sys
import re

from surprise import Dataset
from surprise import Reader

from collections import defaultdict
import numpy as np
from snowflake import connector
import pandas as pd

conn = connector.connect(
    user='kliantono',
    password='ford.SIG7tuch_grar',
    account='ada15167',
    warehouse='COMPUTE_WH',
    database='WITHIN'
)

cur_interactions = conn.cursor()
cur_items = conn.cursor()
cur_ratings = conn.cursor()

interactions_sql = '''
    select
    *
    from (
        select
            ws.USER_ID
            , ws.WORKOUT_ID as item_id
            , date_part('epoch_millisecond', ws.CREATED_AT) as timestamp
            -- Map workout 'starts' to click
            , 'Click' as event_type
            , null as event_value
        from within.pgprodflow.workout_sessions ws
        join within.pgprodflow.workouts w
            on ws.WORKOUT_ID = w.ID
        where ws.CREATED_AT >= '2020-04-23'
            and w.WORKOUT_TYPE in ('classic', 'boxing')
            and w.ID NOT IN (1407, 1405, 1361, 1319, 965)
            and ws.CREATED_AT > w.LAUNCH_DATE
            and ws.user_id NOT IN ('Ua61ZFw', 'Uq6knIA', 'UkgX8SQ', 'UKXQuLg', 'UQM6URw', 'Uyyo4Kw')

        UNION

        select
            ws.USER_ID
            , ws.WORKOUT_ID as item_id
            , date_part('epoch_millisecond', ws.CREATED_AT) as timestamp
            -- Map workout completes to watch
            , 'Watch' as event_type
            , ws.TOTAL_SCORE as event_value
        from within.pgprodflow.workout_sessions ws
        join within.pgprodflow.workouts w
            on ws.WORKOUT_ID = w.ID
        where ws.CREATED_AT >= '2020-04-23'
            and w.WORKOUT_TYPE in ('classic', 'boxing')
            and w.ID NOT IN (1407, 1405, 1361, 1319, 965)
            and ws.CREATED_AT > w.LAUNCH_DATE
            -- use the incomplete flag to determine workout complete
            and ws.INCOMPLETE = false
            and ws.user_id NOT IN ('Ua61ZFw', 'Uq6knIA', 'UkgX8SQ', 'UKXQuLg', 'UQM6URw', 'Uyyo4Kw')
        LIMIT 30000
        ) a

    UNION

    select
    *
    from (
        select
            ws.USER_ID
            , ws.WORKOUT_ID as item_id
            , date_part('epoch_millisecond', ws.CREATED_AT) as timestamp
            -- Map workout 'starts' to click
            , 'Click' as event_type
            , null as event_value
        from within.pgprodflow.workout_sessions ws
        join within.pgprodflow.workouts w
            on ws.WORKOUT_ID = w.ID
        where ws.CREATED_AT >= '2020-04-23'
            and w.WORKOUT_TYPE in ('classic', 'boxing')
            and w.ID NOT IN (1407, 1405, 1361, 1319, 965)
            and ws.CREATED_AT > w.LAUNCH_DATE
            and ws.user_id IN ('Ua61ZFw', 'Uq6knIA', 'UkgX8SQ', 'UKXQuLg', 'UQM6URw', 'Uyyo4Kw')

        UNION

        select
            ws.USER_ID
            , ws.WORKOUT_ID as item_id
            , date_part('epoch_millisecond', ws.CREATED_AT) as timestamp
            -- Map workout completes to watch
            , 'Watch' as event_type
            , ws.TOTAL_SCORE as event_value
        from within.pgprodflow.workout_sessions ws
        join within.pgprodflow.workouts w
            on ws.WORKOUT_ID = w.ID
        where ws.CREATED_AT >= '2020-04-23'
            and w.WORKOUT_TYPE in ('classic', 'boxing')
            and w.ID NOT IN (1407, 1405, 1361, 1319, 965)
            and ws.CREATED_AT > w.LAUNCH_DATE
            -- use the incomplete flag to determine workout complete
            and ws.INCOMPLETE = false
            and ws.user_id IN ('Ua61ZFw', 'Uq6knIA', 'UkgX8SQ', 'UKXQuLg', 'UQM6URw', 'Uyyo4Kw')
        LIMIT 30000
    ) b
'''

items_sql = '''
    WITH genre AS (
        SELECT
            g.ID
            , IFF(g.genres_list != '', g.genres_list, 'No Genre') AS genres
            , IFF(g.collections_list != '', g.collections_list, 'No Collections') AS collections
            , IFF(g.programs_list != '', g.programs_list, 'No Programs') AS programs
            , IFF(g.carousels_list != '', g.carousels_list, 'No Carousels') AS carousels
        FROM ( -- created a subquery to make it easier to read. We need another IFF function outside of the subquery,
            -- because LISTAGG returns am empty STRING if we input NULL values, an we want to replace that will a STRING
            -- declaring that there are e.g. 'No Genre', 'No Collections', etc.
            SELECT
                w.ID
                -- There are multiple rows for the same WORKOUT_ID for each distinct TYPES & VALUE ({genre: pop, rock, collection: decades...}
                -- Two VALUES for the same TYPE (e.g. genre: pop, rock), equates to two rows for the WORKOUT_ID
                -- In order to aggregate the VALUES of the same TYPE into a single row for each WORKOUT_ID, we must use the IFF function
                -- inside the LISTAGG function and provide a NULL value for the ELSE condition (if the WORKOUT_ID is missing the declared TYPE.)
                -- If we substitute the NULL value with a STRING value (e.g.'No genre') for the ELSE condition, it would duplicate the STRING
                -- for the number of rows there are for a WORKOUT_ID (e.g. 'Pop | No genre | No genre', instead of just 'Pop')
                , LISTAGG(IFF(c.TYPE = 'genre', c.NAME, NULL), '|') WITHIN GROUP(ORDER BY c."ORDER") AS genres_list
                , LISTAGG(IFF(c.TYPE = 'collection', c.NAME, NULL), '|') WITHIN GROUP(ORDER BY c."ORDER") AS collections_list
                , LISTAGG(IFF(c.TYPE = 'program', c.NAME, NULL), '|') WITHIN GROUP(ORDER BY c."ORDER") AS programs_list
                , LISTAGG(IFF(c.TYPE = 'carousel', c.NAME, NULL), '|') WITHIN GROUP(ORDER BY c."ORDER") AS carousels_list
            FROM WITHIN.PGPRODFLOW.WORKOUTS w
            LEFT JOIN WITHIN.PGPRODFLOW.CATEGORY_WORKOUTS cw ON cw.WORKOUT_ID = w.ID
            LEFT JOIN WITHIN.PGPRODFLOW.CATEGORIES c ON c.ID = cw.CATEGORY_ID
            WHERE 1=1
                -- Has to be a workout in headset by the current date, no tutorials or demos, only boxing and flow
                AND w.DISABLED != TRUE
                AND w.VISIBILITY = 'public'
                AND w.IS_LIFECYCLE = 0
                AND w.WORKOUT_TYPE in ('classic', 'boxing')
                AND w.ID NOT IN (1407, 1405, 1361, 1319, 965)
                AND w.LAUNCH_DATE <= CURRENT_DATE()
                AND cw.PUBLISHED_VERSION = 1
            GROUP BY 1
            ORDER BY 1
        ) g
    )

    , rating AS (
        -- Normalize ratings to the first 14 days after launch
        SELECT
            r.TABLE_ID,
            AVG(r.RATING) avg_rating
        FROM WITHIN.PGPRODFLOW.RATINGS r
        LEFT JOIN WITHIN.PGPRODFLOW.WORKOUTS w ON w.ID = r.TABLE_ID
        WHERE 1=1
            -- Has to be a workout in headset by the current date, no tutorials or demos, only boxing and flow
            AND r.RATING != 0
            AND r.CREATED_AT < DATEADD('DAY', 14, w.LAUNCH_DATE)
            AND w.LAUNCH_DATE <= CURRENT_DATE()
            AND w.DISABLED != TRUE
            AND w.VISIBILITY = 'public'
            AND w.IS_LIFECYCLE = 0
            AND w.WORKOUT_TYPE in ('classic', 'boxing')
            AND w.ID NOT IN (1407, 1405, 1361, 1319, 965)
        GROUP BY 1
    )

    SELECT
        w.ID AS workoutId -- WORKOUT_ID
        , w.TITLE AS title
        --, ROUND(r.AVG_RATING, 2) AS AVERAGE_RATING -- RATING
        --, DATE_PART('epoch_millisecond', w.LAUNCH_DATE) AS CREATION_TIMESTAMP -- LAUNCH_TIMESTAMP
        --, IFF(wi.NAME IS NOT NULL, wi.NAME, NULL) AS INTENSITY
        --, IFF(g.genres IS NOT NULL, g.genres, NULL) AS genres
        , IFF(g.genres IS NOT NULL, g.genres, 'No genre') AS genres
        --, IFF(g.collections IS NOT NULL, g.collections, NULL) AS COLLECTIONS
    --     , IFF(g.programs IS NOT NULL, g.programs, NULL) AS PROGRAMS
        --, IFF(g.carousels IS NOT NULL, g.carousels, NULL) AS CAROUSELS
        --, w.WORKOUT_TYPE AS CONTENT_CLASSIFICATION -- WORKOUT_TYPE
        --, CONCAT(i.FIRST_NAME, ' ', i.LAST_NAME) AS CONTENT_OWNER -- COACH
        --, ROUND(w.DURATION / 60, 2) AS DURATION -- DURATION
        --, w.DESCRIPTION -- PRODUCT_DESCRIPTION
    FROM WITHIN.PGPRODFLOW.WORKOUTS w
    LEFT JOIN WITHIN.PGPRODFLOW.WORKOUT_INTENSITY wi ON wi.id = w.intensity_id
    LEFT JOIN WITHIN.PGPRODFLOW.INSTRUCTOR i ON i.id = w.instructor_id
    LEFT JOIN rating r ON r.table_id = w.id
    LEFT JOIN genre g ON g.ID = w.ID
    WHERE 1=1
        -- Has to be a workout in headset by the current date, no tutorials or demos, only boxing and flow
        AND w.LAUNCH_DATE <= CURRENT_DATE()
        AND w.DISABLED != TRUE
        AND w.VISIBILITY = 'public'
        AND w.IS_LIFECYCLE = 0
        AND w.WORKOUT_TYPE in ('classic', 'boxing')
        AND w.ID NOT IN (1407, 1405, 1361, 1319, 965)
    ORDER BY 1
'''

ratings_sql = '''
    SELECT
    *
    FROM (
        SELECT
            AUTHOR_ID AS userId
            , TABLE_ID AS workoutId
            , RATING AS rating
            --# , DATE_PART('epoch_millisecond', CREATED_AT) as timestamp
        FROM WITHIN.PGPRODFLOW.RATINGS r
        JOIN WITHIN.PGPRODFLOW.WORKOUTS w ON w.ID = r.TABLE_ID
        WHERE 1=1
            AND r.RATING != 0
            AND w.LAUNCH_DATE <= CURRENT_DATE()
            AND w.DISABLED != TRUE
            AND w.VISIBILITY = 'public'
            AND w.IS_LIFECYCLE = 0
            AND w.WORKOUT_TYPE in ('classic', 'boxing')
            AND w.ID NOT IN (1407, 1405, 1361, 1319, 965)
            AND AUTHOR_ID NOT IN ('Ua61ZFw', 'Uq6knIA', 'UkgX8SQ', 'UKXQuLg', 'UQM6URw', 'Uyyo4Kw')
        LIMIT 30000
    ) a

    UNION

    SELECT
    *
    FROM (
        SELECT
            AUTHOR_ID AS userId
            , TABLE_ID AS workoutId
            , RATING AS rating
            --# , DATE_PART('epoch_millisecond', CREATED_AT) as timestamp
        FROM WITHIN.PGPRODFLOW.RATINGS r
        JOIN WITHIN.PGPRODFLOW.WORKOUTS w ON w.ID = r.TABLE_ID
        WHERE 1=1
            AND r.RATING != 0
            AND w.LAUNCH_DATE <= CURRENT_DATE()
            AND w.DISABLED != TRUE
            AND w.VISIBILITY = 'public'
            AND w.IS_LIFECYCLE = 0
            AND w.WORKOUT_TYPE in ('classic', 'boxing')
            AND w.ID NOT IN (1407, 1405, 1361, 1319, 965)
            AND AUTHOR_ID IN ('Ua61ZFw', 'Uq6knIA', 'UkgX8SQ', 'UKXQuLg', 'UQM6URw', 'Uyyo4Kw')
        ORDER BY 1
        LIMIT 30000
    ) b
'''

# cur_interactions.execute(interactions_sql)
# cur_items.execute(items_sql)
# cur_ratings.execute(ratings_sql)

try:
    cur_interactions.execute(interactions_sql)
    all_rows_interactions = cur_interactions.fetchall()
    num_fields_interactions = len(cur_interactions.description)
    field_names_interactions = [i[0] for i in cur_interactions.description]
finally:
    cur_interactions.close()

interactions_df = pd.DataFrame(all_rows_interactions)
interactions_df.columns = field_names_interactions


try:
    cur_items.execute(items_sql)
    all_rows_items = cur_items.fetchall()
    num_fields_items = len(cur_items.description)
    field_names_items = [i[0] for i in cur_items.description]
finally:
    cur_items.close()

items_df = pd.DataFrame(all_rows_items)
items_df.columns = field_names_items


try:
    cur_ratings.execute(ratings_sql)
    all_rows_ratings = cur_ratings.fetchall()
    num_fields_ratings = len(cur_ratings.description)
    field_names_ratings = [i[0] for i in cur_ratings.description]
finally:
    cur_ratings.close()

ratings_df = pd.DataFrame(all_rows_ratings)
ratings_df.columns = field_names_ratings


# interactions_df = cur_interactions.fetch_pandas_all()
# items_df = cur_items.fetch_pandas_all()
# ratings_df = cur_ratings.fetch_pandas_all()

# cur_interactions.close()
# cur_items.close()
# cur_ratings.close()

# interactions_df.to_csv('../ml-latest-small/workout_interactions.csv', header=True)
# items_df.to_csv('../ml-latest-small/workout_items.csv', index=False, header=True)
# ratings_df.to_csv('../ml-latest-small/workout_ratings.csv', index=False, header=True)

# print(interactions_df.head())
# print(items_df.head())
# print(ratings_df.head())

# print(df.dtypes)
# print(df.info())
# print(df.describe())
# print(isinstance(df, pd.DataFrame))

In [ ]:
class WorkoutLens:

    workoutID_to_name = {}
    name_to_workoutID = {}
    ratingsPath = ratings_df
    workoutsPath = items_df

    def loadWorkoutLensLatestSmall(self):

        # Look for files relative to the directory we are running from
        os.chdir(os.path.dirname(sys.argv[0]))

        ratingsDataset = 0
        self.workoutID_to_name = {}
        self.name_to_workoutID = {}

#         reader = Reader(rating_scale=(1, 5))
        reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)


        ratingsDataset = Dataset.load_from_df(self.ratingsPath, reader=reader)

        for row in self.workoutsPath.itertuples():
            # workoutID = int(row['WORKOUTID'])
            # workoutName = row['TITLE']
            workoutID = int(row[1])
            workoutName = row[2]
            self.workoutID_to_name[workoutID] = workoutName
            self.name_to_workoutID[workoutName] = workoutID

        return ratingsDataset

    def getUserRatings(self, user):
        userRatings = []
        hitUser = False

        for row in self.ratingsPath.itertuples():
            # userID = int(row['USERID'])
            userID = int(row[1])
            if (user == userID):
                # workoutID = int(row['WORKOUTID'])
                # rating = float(row['RATING'])
                workoutID = int(row[2])
                rating = float(row[3])
                userRatings.append((workoutID, rating))
                hitUser = True
            if (hitUser and (user != userID)):
                break

        return userRatings

    def getPopularityRanks(self):
        ratings = defaultdict(int)
        rankings = defaultdict(int)

        for row in self.ratingsPath.itertuples():
            # workoutID = int(row['WORKOUTID'])
            workoutID = int(row[2])
            ratings[workoutID] += 1

        rank = 1
        for workoutID, ratingCount in sorted(ratings.items(), key=lambda x: x[1], reverse=True):
            rankings[workoutID] = rank
            rank += 1

        return rankings

    def getGenres(self):
        genres = defaultdict(list)
        genreIDs = {}
        maxGenreID = 0

        for row in self.workoutsPath.itertuples():
            # workoutID = int(row['WORKOUTID'])
            # genreList = row['GENRES'].split('|')
            workoutID = int(row[1])
            genreList = row[3].split('|')
            genreIDList = []
            for genre in genreList:
                if genre in genreIDs:
                    genreID = genreIDs[genre]
                else:
                    genreID = maxGenreID
                    genreIDs[genre] = genreID
                    maxGenreID += 1
                genreIDList.append(genreID)
            genres[workoutID] = genreIDList

        # Convert integer-encoded genre lists to bitfields that we can treat as vectors
        for (workoutID, genreIDList) in genres.items():
            bitfield = [0] * maxGenreID
            for genreID in genreIDList:
                bitfield[genreID] = 1
            genres[workoutID] = bitfield

        return genres

    def getYears(self):
        p = re.compile(r"(?:\((\d{4})\))?\s*$")
        years = defaultdict(int)

        for row in self.workoutsPath.itertuples():
            # workoutID = int(row['WORKOUTID'])
            # title = row['TITLE']
            workoutID = int(row[1])
            title = row[2]
            m = p.search(title)
            year = m.group(1)
            if year:
                years[workoutID] = int(year)

        return years

    def getMiseEnScene(self):
        mes = defaultdict(list)
        with open("LLVisualFeatures13K_Log.csv", newline='') as csvfile:
            mesReader = csv.reader(csvfile)
            next(mesReader)
            for row in mesReader:
                workoutID = int(row[0])
                avgShotLength = float(row[1])
                meanColorVariance = float(row[2])
                stddevColorVariance = float(row[3])
                meanMotion = float(row[4])
                stddevMotion = float(row[5])
                meanLightingKey = float(row[6])
                numShots = float(row[7])
                mes[workoutID] = [avgShotLength, meanColorVariance, stddevColorVariance,
                   meanMotion, stddevMotion, meanLightingKey, numShots]
        return mes

    def getWorkoutName(self, workoutID):
        if workoutID in self.workoutID_to_name:
            return self.workoutID_to_name[workoutID]
        else:
            return ""

    def getWorkoutID(self, workoutName):
        if workoutName in self.name_to_workoutID:
            return self.name_to_workoutID[workoutName]
        else:
            return 0

In [ ]:
print(interactions_df.size)
print(interactions_df.memory_usage(index=True).sum())
print(items_df.size)
print(items_df.memory_usage(index=True).sum())
print(ratings_df.size)
print(ratings_df.memory_usage(index=True).sum())

In [ ]:
interactions_df.head()

In [ ]:
items_df.head()

In [ ]:
ratings_df.head()

In [ ]:
import itertools

from surprise import accuracy
from collections import defaultdict

class RecommenderMetrics:

    def MAE(predictions):
        return accuracy.mae(predictions, verbose=False)

    def RMSE(predictions):
        return accuracy.rmse(predictions, verbose=False)

    def GetTopN(predictions, n=20, minimumRating=4.0):
        topN = defaultdict(list)


        for userID, workoutID, actualRating, estimatedRating, _ in predictions:
            if (estimatedRating >= minimumRating):
                topN[int(userID)].append((int(workoutID), estimatedRating))

        for userID, ratings in topN.items():
            ratings.sort(key=lambda x: x[1], reverse=True)
            topN[int(userID)] = ratings[:n]

        return topN

    def HitRate(topNPredicted, leftOutPredictions):
        hits = 0
        total = 0

        # For each left-out rating
        for leftOut in leftOutPredictions:
            userID = leftOut[0]
            leftOutWorkoutID = leftOut[1]
            # Is it in the predicted top 20 for this user?
            hit = False
            for workoutID, predictedRating in topNPredicted[int(userID)]:
                if (int(leftOutWorkoutID) == int(workoutID)):
                    hit = True
                    break
            if (hit) :
                hits += 1

            total += 1

        # Compute overall precision
        return hits/total

    def CumulativeHitRate(topNPredicted, leftOutPredictions, ratingCutoff=0):
        hits = 0
        total = 0

        # For each left-out rating
        for userID, leftOutWorkoutID, actualRating, estimatedRating, _ in leftOutPredictions:
            # Only look at ability to recommend things the users actually liked...
            if (actualRating >= ratingCutoff):
                # Is it in the predicted top 20 for this user?
                hit = False
                for workoutID, predictedRating in topNPredicted[int(userID)]:
                    if (int(leftOutWorkoutID) == workoutID):
                        hit = True
                        break
                if (hit) :
                    hits += 1

                total += 1

        # Compute overall precision
        return hits/total

    def RatingHitRate(topNPredicted, leftOutPredictions):
        hits = defaultdict(float)
        total = defaultdict(float)

        # For each left-out rating
        for userID, leftOutWorkoutID, actualRating, estimatedRating, _ in leftOutPredictions:
            # Is it in the predicted top N for this user?
            hit = False
            for workoutID, predictedRating in topNPredicted[int(userID)]:
                if (int(leftOutWorkoutID) == workoutID):
                    hit = True
                    break
            if (hit) :
                hits[actualRating] += 1

            total[actualRating] += 1

        # Compute overall precision
        for rating in sorted(hits.keys()):
            print(rating, hits[rating] / total[rating])

    def AverageReciprocalHitRank(topNPredicted, leftOutPredictions):
        summation = 0
        total = 0
        # For each left-out rating
        for userID, leftOutWorkoutID, actualRating, estimatedRating, _ in leftOutPredictions:
            # Is it in the predicted top N for this user?
            hitRank = 0
            rank = 0
            for workoutID, predictedRating in topNPredicted[int(userID)]:
                rank = rank + 1
                if (int(leftOutWorkoutID) == workoutID):
                    hitRank = rank
                    break
            if (hitRank > 0) :
                summation += 1.0 / hitRank

            total += 1

        return summation / total

    # What percentage of users have at least one "good" recommendation
    def UserCoverage(topNPredicted, numUsers, ratingThreshold=0):
        hits = 0
        for userID in topNPredicted.keys():
            hit = False
            for workoutID, predictedRating in topNPredicted[userID]:
                if (predictedRating >= ratingThreshold):
                    hit = True
                    break
            if (hit):
                hits += 1

        return hits / numUsers

    def Diversity(topNPredicted, simsAlgo):
        n = 0
        total = 0
        simsMatrix = simsAlgo.compute_similarities()
        for userID in topNPredicted.keys():
            pairs = itertools.combinations(topNPredicted[userID], 2)
            for pair in pairs:
                workout1 = pair[0][0]
                workout2 = pair[1][0]
                innerID1 = simsAlgo.trainset.to_inner_iid(str(workout1))
                innerID2 = simsAlgo.trainset.to_inner_iid(str(workout2))
                similarity = simsMatrix[innerID1][innerID2]
                total += similarity
                n += 1

        S = total / n
        return (1-S)

    def Novelty(topNPredicted, rankings):
        n = 0
        total = 0
        for userID in topNPredicted.keys():
            for rating in topNPredicted[userID]:
                workoutID = rating[0]
                rank = rankings[workoutID]
                total += rank
                n += 1
        return total / n

In [ ]:
from surprise.model_selection import train_test_split
from surprise.model_selection import LeaveOneOut
from surprise import KNNBaseline

class EvaluationData:

    def __init__(self, data, popularityRankings):

        self.rankings = popularityRankings

        #Build a full training set for evaluating overall properties
        self.fullTrainSet = data.build_full_trainset()
        self.fullAntiTestSet = self.fullTrainSet.build_anti_testset()

        #Build a 75/25 train/test split for measuring accuracy
        self.trainSet, self.testSet = train_test_split(data, test_size=.25, random_state=1)

        #Build a "leave one out" train/test split for evaluating top-N recommenders
        #And build an anti-test-set for building predictions
        LOOCV = LeaveOneOut(n_splits=1, random_state=1)
        for train, test in LOOCV.split(data):
            self.LOOCVTrain = train
            self.LOOCVTest = test

        self.LOOCVAntiTestSet = self.LOOCVTrain.build_anti_testset()

        #Compute similarty matrix between items so we can measure diversity
        sim_options = {'name': 'cosine', 'user_based': False}
        self.simsAlgo = KNNBaseline(sim_options=sim_options)
        self.simsAlgo.fit(self.fullTrainSet)

    def GetFullTrainSet(self):
        return self.fullTrainSet

    def GetFullAntiTestSet(self):
        return self.fullAntiTestSet

    def GetAntiTestSetForUser(self, testSubject):
        trainset = self.fullTrainSet
        fill = trainset.global_mean
        anti_testset = []
        u = trainset.to_inner_uid(str(testSubject))
        user_items = set([j for (j, _) in trainset.ur[u]])
        anti_testset += [(trainset.to_raw_uid(u), trainset.to_raw_iid(i), fill) for
                                 i in trainset.all_items() if
                                 i not in user_items]
        return anti_testset

    def GetTrainSet(self):
        return self.trainSet

    def GetTestSet(self):
        return self.testSet

    def GetLOOCVTrainSet(self):
        return self.LOOCVTrain

    def GetLOOCVTestSet(self):
        return self.LOOCVTest

    def GetLOOCVAntiTestSet(self):
        return self.LOOCVAntiTestSet

    def GetSimilarities(self):
        return self.simsAlgo

    def GetPopularityRankings(self):
        return self.rankings

In [ ]:
# from RecommenderMetrics import RecommenderMetrics
# from EvaluationData import EvaluationData

class EvaluatedAlgorithm:

    def __init__(self, algorithm, name):
        self.algorithm = algorithm
        self.name = name

    def Evaluate(self, evaluationData, doTopN, n=20, verbose=True):
        metrics = {}
        # Compute accuracy
        if (verbose):
            print("Evaluating accuracy...")
        self.algorithm.fit(evaluationData.GetTrainSet())
        predictions = self.algorithm.test(evaluationData.GetTestSet())
        metrics["RMSE"] = RecommenderMetrics.RMSE(predictions)
        metrics["MAE"] = RecommenderMetrics.MAE(predictions)

        if (doTopN):
            # Evaluate top-20 with Leave One Out testing
            if (verbose):
                print("Evaluating top-N with leave-one-out...")
            self.algorithm.fit(evaluationData.GetLOOCVTrainSet())
            leftOutPredictions = self.algorithm.test(evaluationData.GetLOOCVTestSet())
            # Build predictions for all ratings not in the training set
            allPredictions = self.algorithm.test(evaluationData.GetLOOCVAntiTestSet())
            # Compute top 20 recs for each user
            topNPredicted = RecommenderMetrics.GetTopN(allPredictions, n)
            if (verbose):
                print("Computing hit-rate and rank metrics...")
            # See how often we recommended a workout the user actually rated
            metrics["HR"] = RecommenderMetrics.HitRate(topNPredicted, leftOutPredictions)
            # See how often we recommended a workout the user actually liked
            metrics["cHR"] = RecommenderMetrics.CumulativeHitRate(topNPredicted, leftOutPredictions)
            # Compute ARHR
            metrics["ARHR"] = RecommenderMetrics.AverageReciprocalHitRank(topNPredicted, leftOutPredictions)

            #Evaluate properties of recommendations on full training set
            if (verbose):
                print("Computing recommendations with full data set...")
            self.algorithm.fit(evaluationData.GetFullTrainSet())
            allPredictions = self.algorithm.test(evaluationData.GetFullAntiTestSet())
            topNPredicted = RecommenderMetrics.GetTopN(allPredictions, n)
            if (verbose):
                print("Analyzing coverage, diversity, and novelty...")
            # Print user coverage with a minimum predicted rating of 4.0:
            metrics["Coverage"] = RecommenderMetrics.UserCoverage(  topNPredicted,
                                                                   evaluationData.GetFullTrainSet().n_users,
                                                                   ratingThreshold=4.0)
            # Measure diversity of recommendations:
            metrics["Diversity"] = RecommenderMetrics.Diversity(topNPredicted, evaluationData.GetSimilarities())

            # Measure novelty (average popularity rank of recommendations):
            metrics["Novelty"] = RecommenderMetrics.Novelty(topNPredicted,
                                                            evaluationData.GetPopularityRankings())

        if (verbose):
            print("Analysis complete.")

        return metrics

    def GetName(self):
        return self.name

    def GetAlgorithm(self):
        return self.algorithm

In [ ]:
# # from WorkoutLens import WorkoutLens
# from surprise import KNNBasic
# import heapq
# from collections import defaultdict
# from operator import itemgetter
# from surprise.model_selection import LeaveOneOut
# # from RecommenderMetrics import RecommenderMetrics
# # from EvaluationData import EvaluationData

# def LoadWorkoutLensData():
#     ml = WorkoutLens()
#     print("Loading workout ratings...")
#     data = ml.loadWorkoutLensLatestSmall()
#     print("\nComputing workout popularity ranks so we can measure novelty later...")
#     rankings = ml.getPopularityRanks()
#     return (ml, data, rankings)

# ml, data, rankings = LoadWorkoutLensData()

# evalData = EvaluationData(data, rankings)

# # Train on leave-One-Out train set
# trainSet = evalData.GetLOOCVTrainSet()
# sim_options = {'name': 'cosine',
#                'user_based': True
#                }

# model = KNNBasic(sim_options=sim_options)
# model.fit(trainSet)
# simsMatrix = model.compute_similarities()

# leftOutTestSet = evalData.GetLOOCVTestSet()

# # Build up dict to lists of (int(workoutID), predictedrating) pairs
# topN = defaultdict(list)
# k = 10
# for uiid in range(trainSet.n_users):
#     # Get top N similar users to this one
#     similarityRow = simsMatrix[uiid]

#     similarUsers = []
#     for innerID, score in enumerate(similarityRow):
#         if (innerID != uiid):
#             similarUsers.append( (innerID, score) )

#     kNeighbors = heapq.nlargest(k, similarUsers, key=lambda t: t[1])

#     # Get the stuff they rated, and add up ratings for each item, weighted by user similarity
#     candidates = defaultdict(float)
#     for similarUser in kNeighbors:
#         innerID = similarUser[0]
#         userSimilarityScore = similarUser[1]
#         theirRatings = trainSet.ur[innerID]
#         for rating in theirRatings:
#             candidates[rating[0]] += (rating[1] / 5.0) * userSimilarityScore

#     # Build a dictionary of stuff the user has already seen
#     watched = {}
#     for itemID, rating in trainSet.ur[uiid]:
#         watched[itemID] = 1

#     # Get top-rated items from similar users:
#     pos = 0
#     for itemID, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
#         if not itemID in watched:
#             workoutID = trainSet.to_raw_iid(itemID)
#             topN[int(trainSet.to_raw_uid(uiid))].append( (int(workoutID), 0.0) )
#             pos += 1
#             if (pos > 40):
#                 break

# # Measure
# print("HR", RecommenderMetrics.HitRate(topN, leftOutTestSet))

In [ ]:
# User CF

# from WorkoutLens import WorkoutLens
from surprise import KNNBasic
import heapq
from collections import defaultdict
from operator import itemgetter

# testSubject = 'Ua61ZFw' # Dan
# testSubject = 'Uq6knIA' # Mom
# testSubject = 'UkgX8SQ' # Alex
# testSubject = 'UKXQuLg' # Sean
testSubject = 'UQM6URw' # Jenn
# testSubject = 'Uyyo4Kw' # Fharzana
k = 10

# Load our data set and compute the user similarity matrix
ml = WorkoutLens()
data = ml.loadWorkoutLensLatestSmall()

trainSet = data.build_full_trainset()


sim_options = {'name': 'cosine',
               'user_based': True
               }

model = KNNBasic(sim_options=sim_options)
model.fit(trainSet)
simsMatrix = model.compute_similarities()

# Get top N similar users to our test subject
# (Alternate approach would be to select users up to some similarity threshold - try it!)
testUserInnerID = trainSet.to_inner_uid(testSubject)
similarityRow = simsMatrix[testUserInnerID]

similarUsers = []
for innerID, score in enumerate(similarityRow):
    if (innerID != testUserInnerID):
        similarUsers.append( (innerID, score) )

kNeighbors = heapq.nlargest(k, similarUsers, key=lambda t: t[1])

# Get the stuff they rated, and add up ratings for each item, weighted by user similarity
candidates = defaultdict(float)
for similarUser in kNeighbors:
    innerID = similarUser[0]
    userSimilarityScore = similarUser[1]
    theirRatings = trainSet.ur[innerID]
    for rating in theirRatings:
        candidates[rating[0]] += (rating[1] / 5.0) * userSimilarityScore

# Build a dictionary of stuff the user has already seen
watched = {}
for itemID, rating in trainSet.ur[testUserInnerID]:
    watched[itemID] = 1

# Get top-rated items from similar users:
pos = 0
for itemID, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    if not itemID in watched:
        workoutID = trainSet.to_raw_iid(itemID)
        print(ml.getWorkoutID(ml.getWorkoutName(int(workoutID))),ml.getWorkoutName(int(workoutID)),ratingSum)
        pos += 1
        if (pos > 10):
            break

In [ ]:
# Items CF

# from WorkoutLens import WorkoutLens
from surprise import KNNBasic
import heapq
from collections import defaultdict
from operator import itemgetter

# testSubject = 'Ua61ZFw' # Dan
# testSubject = 'Uq6knIA' # Mom
# testSubject = 'UkgX8SQ' # Alex
# testSubject = 'UKXQuLg' # Sean
testSubject = 'UQM6URw' # Jenn
# testSubject = 'Uyyo4Kw' # Fharzana
k = 10

ml = WorkoutLens()
data = ml.loadWorkoutLensLatestSmall()

trainSet = data.build_full_trainset()

sim_options = {'name': 'cosine',
               'user_based': False
               }

model = KNNBasic(sim_options=sim_options)
model.fit(trainSet)
simsMatrix = model.compute_similarities()

testUserInnerID = trainSet.to_inner_uid(testSubject)

# Get the top K items we rated
testUserRatings = trainSet.ur[testUserInnerID]
kNeighbors = heapq.nlargest(k, testUserRatings, key=lambda t: t[1])

# Get similar items to stuff we liked (weighted by rating)
candidates = defaultdict(float)
for itemID, rating in kNeighbors:
    similarityRow = simsMatrix[itemID]
    for innerID, score in enumerate(similarityRow):
        candidates[innerID] += score * (rating / 5.0)

# Build a dictionary of stuff the user has already seen
watched = {}
for itemID, rating in trainSet.ur[testUserInnerID]:
    watched[itemID] = 1

# Get top-rated items from similar users:
pos = 0
for itemID, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    if not itemID in watched:
        workoutID = trainSet.to_raw_iid(itemID)
        print(ml.getWorkoutID(ml.getWorkoutName(int(workoutID))),ml.getWorkoutName(int(workoutID)), ratingSum)
        pos += 1
        if (pos > 10):
            break